### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### **1. Vectorizar documentos**

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [3]:
# Vectoriazación
tfidfvect = TfidfVectorizer()

In [4]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


A continuación se presenta el vocabulario, donde la tupla de coords es la ubicación del documento en el corpus y la ubicación de esta palabra "token" en el diccionario (indice de documento, índice de token). Luego, los valores que se ven a la derecha como Values, son los valores de TF-IDF que representan un valor de aparición del token en documento y penalizando/ajustado por la aparición de este en otros docuemtos, ejemplo tokens como the, is y and tendrían bajo valor de TF-IDF porque a pesar de que aparecen mucho en el documento aparecen mucho en todos los documentos.

In [5]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1103627 stored elements and shape (11314, 101631)>
  Coords	Values
  (0, 95844)	0.2085823901983838
  (0, 97181)	0.11904550931918896
  (0, 48754)	0.08965702221604545
  (0, 18915)	0.14367199533485261
  (0, 68847)	0.058045106782124184
  (0, 88638)	0.05191891673444119
  (0, 30074)	0.06757106887546716
  (0, 37335)	0.17894975360023155
  (0, 60560)	0.056847192482369406
  (0, 68080)	0.08295129558221048
  (0, 88767)	0.16403190525380046
  (0, 25775)	0.4046469916999255
  (0, 80623)	0.11038705606471814
  (0, 88532)	0.1642840263996455
  (0, 68781)	0.06213700528329297
  (0, 31990)	0.09144124612366937
  (0, 51326)	0.07095431715969416
  (0, 34809)	0.1289860835335575
  (0, 84538)	0.14219503554259302
  (0, 57390)	0.11745787957135642
  (0, 89360)	0.03000170182339287
  (0, 21987)	0.04246364738118965
  (0, 41715)	0.10166193380744531
  (0, 55746)	0.12007927287527424
  (0, 9843)	0.1806731975402963
  :	:
  (11313, 40387)	0.0867206848739519
  (11313

Un ejemplo para ver el valor de índice de token de la palabra 'car' en el vocabulario creado a traves del modulo vocabulary_

In [6]:
tfidfvect.vocabulary_['car']

25775

Por último, se genera un diccionario opuesto que va de índices a términos

In [7]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [8]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [9]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

**Similaridad entre documentos**

Se van a seleccionar cinco documentos al azar que tengan más de 8000 caracteres para asegurar un buen contenido en cada uno de estos

In [10]:
doc = []

while len(doc) < 5:
    idx = np.random.randint(0, len(newsgroups_train.data))
    doc.append(idx)

print(doc)


[4490, 5520, 7307, 3287, 7311]


In [11]:
cossim = cosine_similarity(X_train[doc[0]], X_train)[0]
cossim

array([0.05909682, 0.03046645, 0.05139515, ..., 0.016576  , 0.02640338,
       0.02572616], shape=(11314,))

In [12]:
np.sort(cossim)[::-1][1:]

array([0.25238047, 0.23026499, 0.21901442, ..., 0.        , 0.        ,
       0.        ], shape=(11313,))

In [13]:
for i in doc:
    print('*'*50)
    print(f'Índice del documento:\n{i}\nPertenece a la clase:\n{newsgroups_train.target_names[y_train[i]]}')
    # los 5 documentos más similares según coseno:
    cossim = cosine_similarity(X_train[i], X_train)[0]
    print('\n')
    print(f'Top 5 de similaridad por coseno es de {np.round(np.sort(cossim)[::-1][1:6], 2)}')
    # los topics de los 5 documentos más similares:
    mostsim = np.argsort(cossim)[::-1][1:6]
    print('\n')
    print(f'Targets del top 5 de similaridad por coseno')
    for i in mostsim:
        print(newsgroups_train.target_names[y_train[i]])
    print('\n')
    

**************************************************
Índice del documento:
4490
Pertenece a la clase:
rec.sport.baseball


Top 5 de similaridad por coseno es de [0.25 0.23 0.22 0.18 0.16]


Targets del top 5 de similaridad por coseno
rec.sport.baseball
rec.sport.baseball
rec.sport.baseball
rec.sport.baseball
alt.atheism


**************************************************
Índice del documento:
5520
Pertenece a la clase:
talk.religion.misc


Top 5 de similaridad por coseno es de [0.19 0.16 0.16 0.16 0.14]


Targets del top 5 de similaridad por coseno
sci.space
comp.sys.ibm.pc.hardware
sci.space
sci.space
alt.atheism


**************************************************
Índice del documento:
7307
Pertenece a la clase:
rec.sport.hockey


Top 5 de similaridad por coseno es de [0.37 0.35 0.33 0.33 0.3 ]


Targets del top 5 de similaridad por coseno
rec.sport.hockey
rec.sport.hockey
rec.sport.hockey
rec.sport.hockey
rec.sport.hockey


**************************************************
Índice de

En general se observan bajos valores de coseno, esto puede causar que los topics de los documentos encontrados por esta metrica de similitud no sean los mismos al del documento original. Por ende, el coseno puede no ser un buen clasificador de documentos y otras técnicas de clasificación, como árboles de regresión generarían clasificaciones de documentos más precisas.

### **2. Clasificación con Naïve Bayes (NB)**

### Metodología de optimización con NB

El objetivo fundamental de esta fase es la **optimización de hiperparámetros** para maximizar el desempeño predictivo del clasificador Naive Bayes, medido a través del *F1-score macro*. El rendimiento del modelo no solo depende del algoritmo de clasificación en sí, sino también de la etapa previa de vectorización del texto. Por ello, se aborda la sintonización conjunta de los parámetros tanto del *TfidfVectorizer* como del clasificadores bayesianos *MultinomialNB* y *ComplementNB* disponibles en las liberías de *scikit-learn*.

Para llevar a cabo esta optimización, se ha seleccionado un enfoque de **optimización bayesiana**, implementado a través de *BayesSearchCV*. A diferencia de una búsqueda exhaustiva (*Grid Search*), este método es más eficiente en términos de cómputo, ya que construye un modelo probabilístico de la función objetivo (*F1-score*) y lo utiliza para seleccionar de forma inteligente las combinaciones de hiperparámetros más prometedoras a evaluar, convergiendo más rápidamente hacia una solución óptima.


A continuación, se define el **espacio de búsqueda** para este proceso. Dicho espacio está estructurado como un diccionario donde cada entrada corresponde a un hiperparámetro del pipeline (*vect__* para el vectorizador y *clf__* para el clasificador). Para cada uno, se especifica un dominio de valores (real, entero o categórico) y una distribución de probabilidad a priori (*uniform* o *log-uniform*) que refleja las suposiciones iniciales sobre la influencia del parámetro.

### *Hiperparámetros a explorar en el espacio de búsqueda*

**alfa (α):** para no obtener valores de 0 de probabilidad durante el entrenamiento en los terminos que no están en el vocabulario, se ajusta α para evitar este valor de 0. Las *smoothing priors* α ≥ 0 tienen en cuenta las características que no están presentes en las muestras de aprendizaje y evitan probabilidades nulas en cálculos posteriores. Establecer α = 1 se denomina *Laplace smoothing*, mientras que α < 0 se denomina *Lidstone smoothing*. Se evaluan el rango de valores 1e-3 y 1 usando como distribución logaritmo uniforme (*prior*=*'log-uniform'*) para explorar de manera más efectiva los efectos de cambios de magnitud en el hiperparámetro, dando más peso a los valores más pequeños y asegurando una búsqueda más eficiente en un rango amplio de posibles valores

**stop_words:** si es *“english”*, se utiliza una lista de *stop words* incorporada para el inglés. Hay varios problemas conocidos con “english” y se deberían considerar una alternativa (vea Uso de palabras de parada). Si es una *list*, se asume que esa *list* contiene *stop words*, todas las cuales serán eliminadas de los tokens resultantes. Sólo se aplica si analyzer == “word”. Si es *None*, no se utilizarán *stop words*. En este caso, si se establece *max_df* a un valor más alto, como en el rango (0.7, 1.0), se pueden detectar y filtrar automáticamente las *stop words* basándose en la frecuencia de términos dentro del corpus documental. En este caso se evalua *"english"* y *None* con un *max_df* entre 0.7 y 1.0 con *prior*=*'uniform'*, este últimos asume una distribución bayesiana porque no se tiene información sobre la distribución inicial de los parámetros (**'vect__max_df'**).

 
**min_df:** si es *float* varía en el rango entre 0 y 1, sino puede ser un entero (*int*) que por default=1. Cuando se construye el vocabulario, se ignoran los términos que tienen una frecuencia de documentos estrictamente inferior al umbral dado. Este valor también se denomina *cut-off* en la literatura. Si es *float*, el parámetro representa una proporción de documentos, recuentos absolutos enteros. Este parámetro se ignora si el vocabulario no es Ninguno. En este caso se evalúan dos valores enteros, 1 y 5 (**'vect__min_df'**).

**sublinear_tf:** ajusta el conteo de palabras para reducir el impacto de las que se repiten mucho, ayudando a los modelos a enfocarse en la relevancia de una palabra más que en su cantidad bruta. Por ende, se evalúan *True* y *False* para ver si esto mejora la predicción del modelo.


**clf__fit_prior:** este parámetro decide si el modelo debe aprender las probabilidades a priori de las clases directamente de los datos de entrenamiento (*True*), o si debe asumir que todas las clases tienen la misma probabilidad (es decir, probabilidades uniformes) (*False*). Por ende, se evalúa en True y en False.

**clf__norm:** este parámetro es específico del clasificador ComplementNB. Controla si los recuentos de las características (por ejemplo, la frecuencia de las palabras) se normalizan utilizando la norma L1 o L2 antes de que el modelo aplique sus cálculos. Si es *True*, los recuentos de palabras en los documentos se normalizarán. Esto es útil para evitar que documentos muy largos dominen el proceso de decisión simplemente por tener un mayor número de palabras en bruto. Mientras que, si es *False*, se utilizarán los recuentos de palabras sin ninguna normalización.


In [14]:
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [15]:
# Generales
vectorizer_search_space = {
    'vect__max_df': Real(0.7, 1.0, prior='uniform'),
    'vect__min_df': Integer(1, 5),
    'vect__sublinear_tf': Categorical([True, False]),
    'vect__stop_words': Categorical([None, 'english']),
}

# Para el modelo MultinomialNB
mnb_model_space = {
    'clf__alpha': Real(1e-3, 1.0, prior='log-uniform'),
    'clf__fit_prior': Categorical([True, False]),
}

# Para el modelo ComplementNB
cnb_model_space = {
    'clf__alpha': Real(1e-3, 1.0, prior='log-uniform'),
    'clf__fit_prior': Categorical([True, False]),
    'clf__norm': Categorical([True, False]),
}


*MultinomialNB*

In [16]:
search_spaces_mnb = {**vectorizer_search_space, **mnb_model_space}

pipeline_mnb = Pipeline([('vect', TfidfVectorizer()), ('clf', MultinomialNB())])

bayes_search_mnb = BayesSearchCV(
    estimator=pipeline_mnb,
    search_spaces=search_spaces_mnb,
    n_iter=50, cv=5, scoring='f1_macro', n_jobs=-1, verbose=0
)

bayes_search_mnb.fit(newsgroups_train.data, newsgroups_train.target)


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                        ('clf', MultinomialNB())]),
              n_jobs=-1, scoring='f1_macro',
              search_spaces={'clf__alpha': Real(low=0.001, high=1.0, prior='log-uniform', transform='normalize'),
                             'clf__fit_prior': Categorical(categories=(True, False), prior=None),
                             'vect__max_df': Real(low=0.7, high=1.0, prior='uniform', transform='normalize'),
                             'vect__min_df': Integer(low=1, high=5, prior='uniform', transform='normalize'),
                             'vect__stop_words': Categorical(categories=(None, 'english'), prior=None),
                             'vect__sublinear_tf': Categorical(categories=(True, False), prior=None)})

*ComplementNB*

In [17]:

search_spaces_cnb = {**vectorizer_search_space, **cnb_model_space}

pipeline_cnb = Pipeline([('vect', TfidfVectorizer()), ('clf', ComplementNB())])

bayes_search_cnb = BayesSearchCV(
    estimator=pipeline_cnb,
    search_spaces=search_spaces_cnb,
    n_iter=50, cv=5, scoring='f1_macro', n_jobs=-1, verbose=0
)

bayes_search_cnb.fit(newsgroups_train.data, newsgroups_train.target)

BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                        ('clf', ComplementNB())]),
              n_jobs=-1, scoring='f1_macro',
              search_spaces={'clf__alpha': Real(low=0.001, high=1.0, prior='log-uniform', transform='normalize'),
                             'clf__fit_prior': Categorical(categories=(True, False), prior=None),
                             'clf__norm': Categorical(categories=(True, False), prior=None),
                             'vect__max_df': Real(low=0.7, high=1.0, prior='uniform', transform='normalize'),
                             'vect__min_df': Integer(low=1, high=5, prior='uniform', transform='normalize'),
                             'vect__stop_words': Categorical(categories=(None, 'english'), prior=None),
                             'vect__sublinear_tf': Categorical(categories=(True, False), prior=None)})

**Resultados F1-score durante el entrenamiento de los modelos**

In [18]:
print("\nResultados de la optimización")
print(f"Mejor F1-score (CV) para MultinomialNB: {bayes_search_mnb.best_score_:.4f}")
print(f"Mejor F1-score (CV) para ComplementNB: {bayes_search_cnb.best_score_:.4f}")

best_mnb = bayes_search_mnb.best_estimator_
best_cnb = bayes_search_cnb.best_estimator_




Resultados de la optimización
Mejor F1-score (CV) para MultinomialNB: 0.7626
Mejor F1-score (CV) para ComplementNB: 0.7661


**Resultados F1-score sobre los datos de testeo**

In [19]:
print("\nEvaluación en los datos de testeo")
y_pred_mnb = best_mnb.predict(newsgroups_test.data)
test_f1_mnb = f1_score(newsgroups_test.target, y_pred_mnb, average='macro')
print(f"F1-score en Test (MultinomialNB): {test_f1_mnb:.4f}")

y_pred_cnb = best_cnb.predict(newsgroups_test.data)
test_f1_cnb = f1_score(newsgroups_test.target, y_pred_cnb, average='macro')
print(f"F1-score en Test (ComplementNB): {test_f1_cnb:.4f}")


Evaluación en los datos de testeo
F1-score en Test (MultinomialNB): 0.6876
F1-score en Test (ComplementNB): 0.6969


No hubo diferencia entre los modelos de NB, por lo tanto, ambos modelos se pueden usar para clasificar documentos. Por otro lado, si hubieron diferencias entre los modelos en las configuraciones óptimas de los hiperparámetros. Para *MultinomialNB*, el rendimiento óptimo se alcanzó al excluir las stop words en inglés (*stop_words*=*'english'*) y aplicar un suavizado mínimo (α=0.011), lo que sugiere una sensibilidad a términos de alta frecuencia y una alta confianza en los conteos observados. En contraste, *ComplementNB* se fue más preciso con un filtrado dinámico del vocabulario a través de un umbral de frecuencia (*max_df*=0.7), un suavizado mayor (α=0.13) y la inclusión de normalización interna (*norm*). Estas características confieren a *ComplementNB* una mayor robustez frente al ruido léxico y al desequilibrio de clases. Por últmo, Ambos modelos fueron más preciso cuando *fit_prior* = *False*, esto demuestra que la asunción de probabilidades de clase uniformes fue preferible que aprender las proporciones reales de los datos de entrenamiento.

### **3. Similaridad entre palabras usando la transpuesta de la matriz término-documento**

Se invierte la matriz documento-término para obtener una matriz término-documento para calcular la similaridad entre palabras. Por ende, se elijen palabras que sean interpretables y eviten términos aleatorios, en este caso serán *ball*, *doctor*, *python*, *space*, *water*

In [34]:
X_train_transposed = X_train.T
print(f'\nShape de la matriz término-documento: {X_train_transposed.shape}')
print(f'Cantidad de términos (palabras): {X_train_transposed.shape[0]}')
print(f'Cantidad de documentos: {X_train_transposed.shape[1]}')


selected_words = ['ball', 'doctor', 'python', 'space', 'water']
print('\n')
print(f'\nPalabras seleccionadas para analizar la similaridad: {selected_words}\n')

selected_word_indices = []
for word in selected_words:
    if word in tfidfvect.vocabulary_:
        selected_word_indices.append(tfidfvect.vocabulary_[word])
    else:
        print(f"Advertencia: La palabra '{word}' no se encontró en el vocabulario y será omitida.")

print("*"*10 + "Análisis de similaridad entre palabras" + "*"*10)

for i, word_idx in enumerate(selected_word_indices):
    word = selected_words[i]
    word_cossim = cosine_similarity(X_train_transposed[word_idx], X_train_transposed)[0]

    most_similar_word_indices = np.argsort(word_cossim)[::-1][1:6]

    idx_to_word = {idx: w for w, idx in tfidfvect.vocabulary_.items()}

    print(f'\nTop 5 de similaridad por coseno con "{word}":')
    for rank, sim_idx in enumerate(most_similar_word_indices):
        similarity_score = word_cossim[sim_idx]
        similar_word = idx_to_word[sim_idx]
        print(f'{rank+1}. "{similar_word}" (Similaridad: {similarity_score:.4f})')



Shape de la matriz término-documento: (101631, 11314)
Cantidad de términos (palabras): 101631
Cantidad de documentos: 11314



Palabras seleccionadas para analizar la similaridad: ['ball', 'doctor', 'python', 'space', 'water']

**********Análisis de similaridad entre palabras**********

Top 5 de similaridad por coseno con "ball":
1. "runners" (Similaridad: 0.3217)
2. "relieves" (Similaridad: 0.3030)
3. "puzzled" (Similaridad: 0.2935)
4. "runner" (Similaridad: 0.2829)
5. "swing" (Similaridad: 0.2796)

Top 5 de similaridad por coseno con "doctor":
1. "receptionist" (Similaridad: 0.4392)
2. "clinic" (Similaridad: 0.3271)
3. "misbehavior" (Similaridad: 0.2951)
4. "urgent" (Similaridad: 0.2947)
5. "angering" (Similaridad: 0.2943)

Top 5 de similaridad por coseno con "python":
1. "monty" (Similaridad: 0.7138)
2. "bugger" (Similaridad: 0.6743)
3. "sniders" (Similaridad: 0.4863)
4. "mac18" (Similaridad: 0.4863)
5. "comparitively" (Similaridad: 0.4787)

Top 5 de similaridad por coseno con "spa

El análisis de similaridad entre palabras, realizado sobre una matriz transpuesta (término, documento), demuestra con claridad cómo los vectores TF-IDF capturan las relaciones contextuales específicas del corpus. Los resultados varían significativamente según la naturaleza de la palabra, destacándose una muy alta similaridad entre *python* y *monty* (0.71), esto podría inferir algo especifico de estas palabras en algún tipo de contexto. A esto, se evidencia que el modelo identificó la fuerte asociación cultural con el grupo de comedia Monty Python en lugar de solo su uso como lenguaje de programación. Asimismo, se observan fuertes agrupaciones temáticas, como la de *doctor* (médico) con su entorno profesional (*receptionist* (recepcionista), *clinic* (clínica)) y la de *water* (agua) con un contexto de infraestructura urbana (*towers* (torres), *dpw* (sigla de *Department of Public Works* o Departamento de Obras Públicas), *croton* (en referencia al acueducto de Nueva York)). Otras relaciones lógicas pero más generales, como la de *space* (espacio) con *nasa* y la de *ball* (pelota) con términos deportivos como *runners* (corredores), presentan puntajes más moderados. En conjunto, los resultados confirman que esta técnica es efectiva para descubrir no solo el significado literal, sino también las conexiones temáticas, culturales y contextuales subyacentes en los datos.
